In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
"""
Testing RL agent which randomly chooses actions
"""
import sys
sys.path.append('../')
from typing import List
import numpy as np
import os
import pandas as pd
import gym
from envs.battery_storage import BatteryStorageEnv
from tqdm import tqdm
import matplotlib.pyplot as plt

env = BatteryStorageEnv()

episodes = 100

rewards_lst_1 = []

for i in tqdm(range(episodes)):
    ob = env.reset()
    done = False
    rewards = np.zeros(env.MAX_STEPS_PER_EPISODE)

    while not done:
        # random action as policy
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        rewards[env.count - 1] = reward
    rewards_lst_1.append(np.sum(rewards))

# plot episode # versus total episode reward
plt.bar(list(range(episodes)), rewards_lst_1, width=0.5)

# naming the x axis 
plt.xlabel('episode #') 
# naming the y axis 
plt.ylabel('total reward')

plt.show()

In [ ]:
"""
Testing RL agent which chooses charging versus discharging based on the current price
in relation to the running average price.
"""
import sys
sys.path.append('../')
from typing import List
import numpy as np
import os
import pandas as pd
import gym
from envs.battery_storage import BatteryStorageEnv
from tqdm import tqdm
import matplotlib.pyplot as plt

env = BatteryStorageEnv()

episodes = 100

rewards_lst_2 = []

for i in tqdm(range(episodes)):
    ob = env.reset(options = {'reward': 1})
    done = False
    start = True
    rewards = np.zeros(env.MAX_STEPS_PER_EPISODE)
    avg = np.zeros(1)

    while not done:
        if start:
            action = env.action_space.sample()
            state, reward, done, info = env.step(action)
            avg = info['running_avg']
            start = False
        else:
            if env.curr_price >= avg:
                action = np.random.uniform(env.action_space.low, 0.0)
            else:
                action = np.random.uniform(0.0, env.action_space.high)
            state, reward, done, info = env.step(action)
            avg = info['running_avg']

        rewards[env.count - 1] = reward
    rewards_lst_2.append(np.sum(rewards))

# plot episode # versus total episode reward
plt.bar(list(range(episodes)), rewards_lst_2)

# naming the x axis 
plt.xlabel('episode #') 
# naming the y axis 
plt.ylabel('total reward')

plt.show()

In [ ]:
"""
Testing environment with stable_baselines3 library
"""

from stable_baselines3.common.env_checker import check_env

env = BatteryStorageEnv()

check_env(env)

In [ ]:
from stable_baselines3.ddpg.policies import MlpPolicy
from stable_baselines3.common.noise import OrnsteinUhlenbeckActionNoise
from stable_baselines3 import DDPG

env = BatteryStorageEnv()

# the noise objects for DDPG
# n_actions = env.action_space.shape[-1]
# action_noise = OrnsteinUhlenbeckActionNoise(mean=np.zeros(n_actions), sigma=float(0.5) * np.ones(n_actions))

# model = DDPG(MlpPolicy, env, action_noise=action_noise, verbose=1)
# model.learn(total_timesteps=200000)
# model.save("ddpg_simple_battery_env")

# del model # remove to demonstrate saving and loading

model = DDPG.load("ddpg_simple_battery_env")

episodes = 100

rewards_lst_3 = []

for i in tqdm(range(episodes)):
    obs = env.reset()
    done = False
    start = True
    rewards = np.zeros(env.MAX_STEPS_PER_EPISODE)
    avg = np.zeros(1)
    while not done:
        action, _states = model.predict(obs)
        obs, reward, done, info = env.step(action)
        rewards[env.count - 1] = reward
    rewards_lst_3.append(np.sum(rewards))

# plot episode # versus total episode reward
plt.bar(list(range(episodes)), rewards_lst_3)

# naming the x axis 
plt.xlabel('episode #') 
# naming the y axis 
plt.ylabel('total reward')

plt.show()


In [ ]:
width = 0.25
id = np.arange(episodes)
# plot episode # versus total episode reward
plt.bar(id, rewards_lst_1, width, label='random action')
plt.bar(id + width, rewards_lst_2, width, label='running average')
plt.bar(id + 2*width, rewards_lst_3, width, label='DDPG')

# naming the x axis 
plt.xlabel('episode #') 
# naming the y axis 
plt.ylabel('total reward')

plt.legend()

plt.show()